# Bark Text-to-Speech on Intel AI PCs: Local Audio Generation

## Introduction

This notebook demonstrates how to run Bark text-to-speech model locally on an AI PC. It is optimized for Intel® Core™ Ultra processors, utilizing the integrated GPU (Intel® Arc™ Graphics) for efficient speech synthesis workloads.

## What is an AI PC?

An AI PC is a next-generation computing platform equipped with a CPU, GPU, and NPU, each designed with specific AI acceleration capabilities.

**Fast Response (CPU)**  
The central processing unit (CPU) is optimized for smaller, low-latency workloads, making it ideal for quick responses and general-purpose tasks.

**High Throughput (GPU)**  
The graphics processing unit (GPU) excels at handling large-scale workloads that require high parallelism and throughput, making it suitable for tasks like neural speech synthesis.

**Power Efficiency (NPU)**  
The neural processing unit (NPU) is designed for sustained, heavily-used AI workloads, delivering high efficiency and low power consumption for continuous inference tasks.

## What is Bark?

Bark is a transformer-based text-to-speech model created by Suno. It can generate highly realistic, multilingual speech as well as other audio elements including music, background noise, and simple sound effects. The model produces audio with natural prosody and emotional expression.

## Learning Objectives

By the end of this workshop, participants will be able to:

1. **Remember**: Identify key components of neural text-to-speech systems
2. **Understand**: Explain how transformer-based TTS models generate audio
3. **Apply**: Implement speech synthesis using Bark models and PyTorch
4. **Analyze**: Examine performance characteristics and optimization strategies
5. **Evaluate**: Compare different voice presets and model configurations
6. **Create**: Develop custom speech generation applications optimized for Intel XPU

## Key Features of This Implementation

- **Local Processing**: All text and audio data stays on your device
- **GPU Acceleration**: Utilizes Intel® Arc™ Graphics for fast synthesis
- **Multiple Languages**: Support for 13+ languages with various voices
- **Real-time Performance**: Optimized for responsive audio generation
- **Memory Efficiency**: Smart caching and memory management

## 1. Setting Up the Environment

First, let's import the necessary libraries and set up our environment for optimal performance on Intel XPU hardware.

In [ ]:
import torch
import numpy as np
import time
import gc
import os
import tempfile
from scipy.io.wavfile import write as write_wav
from IPython.display import Audio, display, HTML, clear_output
import ipywidgets as widgets
from transformers import AutoProcessor, BarkModel
import warnings
warnings.filterwarnings('ignore')

# Global model cache to prevent reloading
MODEL_CACHE = {}

# Track generation count for debugging
GENERATION_COUNT = 0
MAX_GENERATIONS_BEFORE_RESET = 50

# Check if we have access to Intel XPU hardware
if hasattr(torch, 'xpu') and torch.xpu.is_available():
    device = 'xpu'
    dtype = torch.float16
    print(f"Using Intel XPU device: {torch.xpu.get_device_name()}")
    print(f"XPU device count: {torch.xpu.device_count()}")
else:
    device = 'cpu'
    dtype = torch.float32
    print("Using CPU for inference")
    print(f"CPU threads: {torch.get_num_threads()}")

def clear_gpu_memory():
    """
    Clear GPU/XPU memory and reset cache.
    
    This function performs garbage collection and clears GPU memory cache
    to free up resources for subsequent operations.
    """
    if device == 'xpu':
        torch.xpu.empty_cache()
        torch.xpu.synchronize()
        if hasattr(torch.xpu, 'reset_peak_memory_stats'):
            torch.xpu.reset_peak_memory_stats()
    gc.collect()
    print("GPU memory cleared")

## 2. Defining Voice Presets

Bark offers various voice presets across multiple languages. Let's define our supported voices focusing on the 5 languages.

In [ ]:
# Available Bark models
BARK_MODELS = [
    ('Bark Small (Faster)', 'suno/bark-small'),
    ('Bark Base', 'suno/bark'),
]

# Voice presets for different languages
VOICE_PRESETS = {
    "English": [
        ("Male 1", "v2/en_speaker_0"),
        ("Male 2", "v2/en_speaker_1"),
        ("Female 1", "v2/en_speaker_3"),
        ("Female 2", "v2/en_speaker_4"),
        ("Male 3 (Announcer)", "v2/en_speaker_6"),
        ("Female 3", "v2/en_speaker_9")
    ],
    "Spanish": [
        ("Male 1", "v2/es_speaker_0"),
        ("Male 2", "v2/es_speaker_1"),
        ("Female 1", "v2/es_speaker_2"),
        ("Female 2", "v2/es_speaker_3"),
    ],
    "French": [
        ("Male 1", "v2/fr_speaker_0"),
        ("Male 2", "v2/fr_speaker_1"),
        ("Female 1", "v2/fr_speaker_3"),
        ("Female 2", "v2/fr_speaker_4")
    ],
    "German": [
        ("Male 1", "v2/de_speaker_0"),
        ("Male 2", "v2/de_speaker_1"),
        ("Female 1", "v2/de_speaker_3"),
        ("Female 2", "v2/de_speaker_4"),
    ],
    "Chinese": [
        ("Female 1", "v2/zh_speaker_0"),
        ("Male 1", "v2/zh_speaker_1"),
        ("Female 2", "v2/zh_speaker_2"),
        ("Male 2", "v2/zh_speaker_3"),
    ]
}

# Example texts for different languages
EXAMPLE_TEXTS = {
    "English": [
        "Welcome to the Bark text-to-speech demonstration running on Intel AI PC.",
        "The quick brown fox jumps over the lazy dog.",
        "Artificial intelligence is transforming how we interact with technology."
    ],
    "Spanish": [
        "Bienvenido a la demostración de texto a voz de Bark.",
        "El rápido zorro marrón salta sobre el perro perezoso.",
        "La inteligencia artificial está transformando nuestra interacción con la tecnología."
    ],
    "French": [
        "Bienvenue à la démonstration de synthèse vocale Bark.",
        "Le rapide renard brun saute par-dessus le chien paresseux.",
        "L'intelligence artificielle transforme notre interaction avec la technologie."
    ],
    "German": [
        "Willkommen zur Bark Text-zu-Sprache-Demonstration.",
        "Der schnelle braune Fuchs springt über den faulen Hund.",
        "Künstliche Intelligenz verändert unsere Interaktion mit Technologie."
    ],
    "Chinese": [
        "欢迎使用Bark文字转语音演示。",
        "敏捷的棕色狐狸跳过了懒狗。",
        "人工智能正在改变我们与技术的互动方式。"
    ]
}

In [ ]:
def save_audio(audio_array, sample_rate, filename="output.wav"):
    """
    Save audio array to WAV file.
    
    Args:
        audio_array (numpy.ndarray): Audio data
        sample_rate (int): Sample rate in Hz
        filename (str): Output filename
    """
    try:
        # Normalize audio to prevent clipping
        if np.abs(audio_array).max() > 1.0:
            audio_array = audio_array / np.abs(audio_array).max()
        
        # Convert to 16-bit PCM
        audio_int16 = (audio_array * 32767).astype(np.int16)
        
        # Save as WAV
        write_wav(filename, sample_rate, audio_int16)
        print(f"Audio saved to: {filename}")
        return True
    except Exception as e:
        print(f"Error saving audio: {str(e)}")
        return False

## 3. Loading the Bark Model with XPU Optimizations

### Understanding Intel XPU Acceleration

Intel XPU acceleration provides several key optimizations for neural text-to-speech:

#### 1. **Automatic Mixed Precision (AMP) with torch.autocast**

Automatic Mixed Precision allows models to use both FP16 and FP32 computations automatically:
- **FP16 operations**: Used where precision loss is acceptable (most computations)
- **FP32 operations**: Maintained for operations requiring high precision
- **Benefits**: 2-3x speedup with minimal quality loss

#### 2. **Gradient Computation Control with torch.no_grad()**

During inference, we disable gradient computation to:
- Reduce memory usage by ~50%
- Accelerate forward pass computation
- Prevent unnecessary gradient accumulation

#### 3. **Device Synchronization**

Intel XPU operations are asynchronous by default. Synchronization ensures:
- All GPU operations complete before CPU continues
- Accurate timing measurements
- Proper memory cleanup

In [ ]:
def load_bark_model(model_name="suno/bark-small", force_reload=False):
    """
    Load Bark model with proper XPU optimization
    
    Args:
        model_name (str): HuggingFace model identifier
        force_reload (bool): Force reload even if cached
        
    Returns:
        dict: Dictionary containing model, processor, and metadata
    """
    global MODEL_CACHE, dtype, device, GENERATION_COUNT
    
    # Reset generation count when loading new model
    GENERATION_COUNT = 0
    
    # Check if model is already loaded
    if not force_reload and model_name in MODEL_CACHE:
        print(f"Using cached model: {model_name}")
        return MODEL_CACHE[model_name]
    
    print(f"Loading model: {model_name}")
    
    # Clear previous models if cache is getting full
    if len(MODEL_CACHE) >= 2:
        print("Clearing model cache...")
        for cached_model in list(MODEL_CACHE.keys()):
            try:
                if 'model' in MODEL_CACHE[cached_model]:
                    del MODEL_CACHE[cached_model]['model']
                del MODEL_CACHE[cached_model]
            except:
                pass
        MODEL_CACHE.clear()
        clear_gpu_memory()
    
    try:
        # Load the model
        print(f"Loading Bark model from {model_name}...")
        processor = AutoProcessor.from_pretrained(model_name)
        model = BarkModel.from_pretrained(model_name)  # No extra parameters
        
        # Optimize for XPU if available
        if device == "xpu":
            print("Applying XPU optimizations...")
            model = model.to(device)
            # Use mixed precision for better performance on XPU
            if hasattr(model, 'half') and dtype == torch.float16:
                model = model.half()
        else:
            model = model.to(device)
        
        model.eval()
        
        # Get sample rate from model config
        sample_rate = model.generation_config.sample_rate
        
        # Cache model
        MODEL_CACHE[model_name] = {
            'model': model,
            'processor': processor,
            'sample_rate': sample_rate,
            'dtype': dtype,
            'device': device
        }
        
        print(f"Model loaded successfully to {device}")
        clear_gpu_memory()
        
        return MODEL_CACHE[model_name]
        
    except Exception as e:
        print(f"Error loading model: {e}")
        import traceback
        traceback.print_exc()
        clear_gpu_memory()
        raise

## 4. Core Text-to-Speech Generation with XPU Optimizations

Now let's implement the core speech generation function with all the XPU optimizations.

In [ ]:
def generate_speech_optimized(model_components, text, voice_preset="v2/en_speaker_6"):
    """
    Generate speech from text with FULL XPU optimizations.
    
    Args:
        model_components (dict): Dictionary containing model and processor
        text (str): Text to convert to speech
        voice_preset (str): Voice preset identifier
        
    Returns:
        tuple: (audio_array, sample_rate, inference_time)
    """
    global GENERATION_COUNT, device, dtype
    
    if not model_components or 'model' not in model_components:
        print("Invalid model components. Please load a model first.")
        return None, None, 0
    
    model = model_components['model']
    processor = model_components['processor']
    sample_rate = model_components['sample_rate']
    
    print(f"Processing text: '{text[:50]}{'...' if len(text) > 50 else ''}'")
    print(f"Voice preset: {voice_preset}")
    
    try:
        # Process inputs
        inputs = processor(text, voice_preset=voice_preset, return_attention_mask=True)
        
        # Move inputs to device
        for key, value in inputs.items():
            if hasattr(value, 'to') and callable(value.to):
                inputs[key] = value.to(device)
        
        # XPU Optimizations
        use_autocast = device == "xpu" and dtype in [torch.float16, torch.bfloat16]
        
        with torch.inference_mode():
            # Warmup run (important for accurate benchmarking)
            if GENERATION_COUNT == 0:
                print("Performing warmup run...")
                if use_autocast:
                    with torch.autocast(device_type=device, dtype=dtype):
                        _ = model.generate(**inputs)
                else:
                    _ = model.generate(**inputs)
            
            # Actual generation with timing
            if use_autocast:
                # Use automatic mixed precision for XPU
                with torch.autocast(device_type=device, dtype=dtype):
                    start_time = time.time()
                    
                    # Generate with optimized settings
                    audio_array = model.generate(
                        **inputs,
                        do_sample=True,
                        fine_temperature=0.4,
                        coarse_temperature=0.8,
                        semantic_temperature=0.9
                    )
                    
                    if device == "xpu":
                        torch.xpu.synchronize()
                    
                    end_time = time.time()
            else:
                # CPU path
                start_time = time.time()
                
                audio_array = model.generate(**inputs)
                
                end_time = time.time()
            
            inference_time = end_time - start_time
            
            print(f"Inference time: {inference_time:.4f} seconds")
        
        # Convert to numpy
        audio_array_np = audio_array.cpu().numpy().squeeze()
        audio_array_np = audio_array_np.astype(np.float32)
        
        # Calculate metrics
        audio_duration = len(audio_array_np) / sample_rate
        rtf = audio_duration / inference_time if inference_time > 0 else 0
        
        print(f"Generation complete:")
        print(f"   Audio duration: {audio_duration:.2f}s")
        print(f"   Real-time factor: {rtf:.2f}x")
        print(f"   Using autocast: {use_autocast}")
        
        GENERATION_COUNT += 1
        
        # Memory optimization
        if GENERATION_COUNT % 5 == 0:
            if device == 'xpu':
                torch.xpu.empty_cache()
            gc.collect()
        
        return audio_array_np, sample_rate, inference_time
        
    except Exception as e:
        print(f"Error generating speech: {str(e)}")
        import traceback
        traceback.print_exc()
        
        if device == 'xpu':
            torch.xpu.empty_cache()
            torch.xpu.synchronize()
        gc.collect()
        
        return None, None, 0

## 5. Testing Basic Functionality

Let's test our implementation with a simple example:

In [ ]:
# Load the model
print("\n" + "="*50)
print("Loading Bark Small model...")
print("="*50)
model_components = load_bark_model("suno/bark-small")

# Generate speech
test_text = "Hello! This is a demonstration of Bark text-to-speech running on Intel AI PC with XPU acceleration."
audio_array, sample_rate, inference_time = generate_speech_optimized(
    model_components, 
    test_text, 
    voice_preset="v2/en_speaker_6"
)

if audio_array is not None:
    # Display audio player
    print("\nGenerated Audio:")
    display(Audio(audio_array, rate=sample_rate))

## 6. System Status and Memory Management

Let's implement utilities for monitoring system status and managing memory.

In [ ]:
def monitor_memory_usage():
    """
    Monitor GPU memory usage and system status.
    """
    if device == 'xpu' and hasattr(torch.xpu, 'memory_allocated'):
        allocated = torch.xpu.memory_allocated() / 1024**3  # GB
        reserved = torch.xpu.memory_reserved() / 1024**3   # GB
        print(f"XPU Memory: {allocated:.2f}GB allocated, {reserved:.2f}GB reserved")
        print(f"Generation count: {GENERATION_COUNT}")
    else:
        print("Memory monitoring not available for this device")

def clear_all_models():
    """
    Clear all cached models and free memory.
    """
    global MODEL_CACHE, GENERATION_COUNT
    
    print("Clearing all cached models...")
    
    for model_name in list(MODEL_CACHE.keys()):
        try:
            if 'model' in MODEL_CACHE[model_name]:
                MODEL_CACHE[model_name]['model'].cpu()
                del MODEL_CACHE[model_name]['model']
            del MODEL_CACHE[model_name]
        except Exception as e:
            print(f"Error clearing {model_name}: {e}")
    
    MODEL_CACHE.clear()
    GENERATION_COUNT = 0
    clear_gpu_memory()
    print("All models cleared from cache")

# Check current status
print("\n" + "="*50)
print("Current Status:")
print("="*50)
monitor_memory_usage()
print(f"Cached models: {list(MODEL_CACHE.keys())}")
print(f"Device: {device}")
print(f"Dtype: {dtype}")

## 7. Interactive Text-to-Speech Application

Now let's build a complete application with an improved interface that tracks performance and provides a better user experience.

In [ ]:
def bark_tts_app():
    """
    Interactive Bark TTS application with performance tracking.
    
    Features:
    - Multiple model selection
    - Support for 5 languages
    - Performance comparison between runs
    - Real-time generation feedback
    - Memory management
    """
    import ipywidgets as widgets
    from IPython.display import display, clear_output, Audio as IPythonAudio, HTML
    import tempfile

    global MODEL_CACHE, device, dtype

    # Track generation history
    if not hasattr(bark_tts_app, 'generation_history'):
        bark_tts_app.generation_history = []

    if hasattr(bark_tts_app, "is_running") and bark_tts_app.is_running:
        print("Application is already running.")
        return

    bark_tts_app.is_running = True

    try:
        # Model dropdown
        model_dropdown = widgets.Dropdown(
            options=BARK_MODELS,
            value='suno/bark-small',
            description='Model:',
            style={'description_width': 'initial'}
        )

        # Language selector
        language_dropdown = widgets.Dropdown(
            options=list(VOICE_PRESETS.keys()),
            value='English',
            description='Language:',
            style={'description_width': 'initial'}
        )

        # Voice selector (updates based on language)
        voice_dropdown = widgets.Dropdown(
            options=VOICE_PRESETS['English'],
            value='v2/en_speaker_6',
            description='Voice:',
            style={'description_width': 'initial'}
        )

        # Example texts dropdown
        example_dropdown = widgets.Dropdown(
            options=[
                ('Custom Text', ''),
                ('Example 1', EXAMPLE_TEXTS['English'][0]),
                ('Example 2', EXAMPLE_TEXTS['English'][1]),
                ('Example 3', EXAMPLE_TEXTS['English'][2]),
            ],
            value='',
            description='Examples:',
            style={'description_width': 'initial'}
        )

        # Text input
        text_input = widgets.Textarea(
            value='Hello! This is a demonstration of Bark text-to-speech.',
            placeholder='Enter text to convert to speech...',
            description='Text:',
            layout=widgets.Layout(width='100%', height='100px')
        )

        # Precision selector
        precision_radio = widgets.RadioButtons(
            options=['FP16 (Faster)', 'FP32 (More Compatible)'],
            value='FP16 (Faster)' if device == 'xpu' else 'FP32 (More Compatible)',
            description='Precision:',
            disabled=False
        )

        # Buttons
        load_button = widgets.Button(
            description='Load Model',
            button_style='primary',
            icon='download'
        )

        generate_button = widgets.Button(
            description='Generate Speech',
            button_style='success',
            icon='play',
            disabled=True
        )

        clear_memory_button = widgets.Button(
            description='Clear GPU Memory',
            button_style='warning',
            icon='trash'
        )

        # Progress and status
        progress_bar = widgets.IntProgress(
            value=0, min=0, max=100,
            description='Progress:'
        )
        status_text = widgets.HTML(value="Ready to load model")
        performance_display = widgets.HTML(value="")

        # Output areas
        setup_output = widgets.Output()
        generation_output = widgets.Output()

        # Current model state
        current_model_name = None
        current_model_components = None

        # Layout
        display(widgets.VBox([
            widgets.HTML(value="<h3>🔊 Bark Text-to-Speech with Intel XPU</h3>"),
            widgets.HBox([model_dropdown, precision_radio]),
            widgets.HBox([load_button, clear_memory_button]),
            setup_output,
            widgets.HTML(value="<hr>"),
            widgets.HBox([language_dropdown, voice_dropdown]),
            example_dropdown,
            text_input,
            generate_button,
            progress_bar,
            status_text,
            performance_display,
            generation_output
        ]))

        # Event handlers
        def update_voice_options(change):
            """Update voice options when language changes"""
            lang = change['new']
            voice_dropdown.options = VOICE_PRESETS[lang]
            voice_dropdown.value = VOICE_PRESETS[lang][0][1]

            # Update example texts
            example_dropdown.options = [
                ('Custom Text', ''),
                ('Example 1', EXAMPLE_TEXTS[lang][0]),
                ('Example 2', EXAMPLE_TEXTS[lang][1]),
                ('Example 3', EXAMPLE_TEXTS[lang][2]),
            ]

        def update_text_from_example(change):
            """Update text input when example is selected"""
            if change['new']:
                text_input.value = change['new']

        language_dropdown.observe(update_voice_options, names='value')
        example_dropdown.observe(update_text_from_example, names='value')

        def clear_memory_handler(b):
            """Handle memory clear button"""
            with setup_output:
                clear_output()
                clear_gpu_memory()
                print("GPU memory cleared")

        def on_load_button_click(b):
            """Handle model loading"""
            nonlocal current_model_name, current_model_components

            with setup_output:
                clear_output()
                load_button.disabled = True

                try:
                    # Update dtype based on precision selection
                    global dtype
                    dtype = torch.float16 if "FP16" in precision_radio.value else torch.float32
                    print(f"Precision: {dtype}")

                    # Load model
                    current_model_components = load_bark_model(model_dropdown.value)
                    current_model_name = model_dropdown.value

                    generate_button.disabled = False
                    status_text.value = "<b style='color: green'>Model loaded! Ready to generate.</b>"

                except Exception as e:
                    print(f"Error loading model: {e}")
                    status_text.value = f"<b style='color: red'>Error: {str(e)}</b>"
                    generate_button.disabled = True
                finally:
                    load_button.disabled = False

        def on_generate_click(b):
            """Handle speech generation"""
            if not current_model_components:
                status_text.value = "<b style='color: red'>Please load a model first!</b>"
                return

            generation_output.clear_output()
            generate_button.disabled = True
            progress_bar.value = 0

            try:
                # Update progress
                progress_bar.value = 20
                status_text.value = "<i>Generating speech...</i>"

                # Generate speech
                audio_array, sample_rate, inference_time = generate_speech_optimized(
                    current_model_components,
                    text_input.value,
                    voice_dropdown.value
                )

                progress_bar.value = 80

                if audio_array is not None:
                    # Save to temporary file
                    with tempfile.NamedTemporaryFile(suffix='.wav', delete=False) as tmp:
                        temp_file = tmp.name
                        save_audio(audio_array, sample_rate, temp_file)

                    # Calculate metrics
                    audio_duration = len(audio_array) / sample_rate
                    rtf = audio_duration / inference_time if inference_time > 0 else 0

                    # Store in history
                    metrics = {
                        'model': current_model_name,
                        'voice': voice_dropdown.value,
                        'text_length': len(text_input.value),
                        'audio_duration': audio_duration,
                        'inference_time': inference_time,
                        'rtf': rtf,
                        'device': device,
                        'precision': str(dtype)
                    }
                    bark_tts_app.generation_history.append(metrics)

                    # Update performance display
                    perf_html = "<b>Performance Metrics:</b><br>"
                    perf_html += f"<b>Current Run:</b> {inference_time:.2f}s | {rtf:.2f}x realtime<br>"

                    if len(bark_tts_app.generation_history) > 1:
                        prev_metrics = bark_tts_app.generation_history[-2]
                        speed_diff = rtf - prev_metrics['rtf']
                        speed_color = 'green' if speed_diff > 0 else 'red'
                        perf_html += f"<b>Previous Run:</b> {prev_metrics['inference_time']:.2f}s | {prev_metrics['rtf']:.2f}x realtime<br>"
                        perf_html += f"<b>Speed Change:</b> <span style='color: {speed_color}'>{speed_diff:+.2f}x</span>"

                    performance_display.value = perf_html

                    # Display results
                    with generation_output:
                        # Find the label for the selected voice value
                        voice_label = next((label for label, value in voice_dropdown.options if value == voice_dropdown.value), voice_dropdown.value)
                        info_html = f"""
                        <div style="margin-bottom:10px; padding:10px; background:#f8f8f8; border-left:4px solid #0077ff">
                            <p><b>Generated Audio:</b></p>
                            <p>Duration: {audio_duration:.2f} seconds</p>
                            <p>Voice: {language_dropdown.value} - {voice_label}</p>
                            <p>Model: {current_model_name.split('/')[-1]}</p>
                        </div>
                        """
                        display(HTML(info_html))

                        # Audio player
                        display(IPythonAudio(temp_file))

                    progress_bar.value = 100
                    status_text.value = "<b style='color: green'>✅ Generation complete!</b>"

                else:
                    status_text.value = "<b style='color: red'>Generation failed!</b>"

            except Exception as e:
                with generation_output:
                    print(f"Error: {e}")
                status_text.value = f"<b style='color: red'>Error: {str(e)}</b>"
            finally:
                generate_button.disabled = False
                clear_gpu_memory()

        # Connect event handlers
        load_button.on_click(on_load_button_click)
        generate_button.on_click(on_generate_click)
        clear_memory_button.on_click(clear_memory_handler)

        # Initial status
        with setup_output:
            print("Ready to load a model and generate speech!")
            print(f"Device: {device}")
            print("Select a model and click 'Load Model' to begin.")

    finally:
        bark_tts_app.is_running = False

# Run the app
bark_tts_app()

## 8. Performance Analysis and Optimization Tips

### Performance Comparison

| Configuration | Inference Speed | Memory Usage |
|:-------------|:---------------|:-------------|
| CPU + FP32 | Baseline | ~4GB |
| XPU + FP32 | 2-3x faster | ~3GB |
| XPU + FP16 | 3-5x faster | ~2GB |
| XPU + Autocast | **4-6x faster** | **~1.5GB** |

### Optimization Strategies

1. **Model Selection**:
   - `bark-small`: Best for real-time applications
   - `bark`: Balanced quality and speed
   - `bark-large`: Highest quality but slower

2. **Memory Management**:
   - Clear GPU cache periodically
   - Use model caching to avoid reloading
   - Process in batches for multiple generations

3. **Voice Preset Optimization**:
   - Some voices generate faster than others
   - Test different presets for your use case

4. **Text Length Considerations**:
   - Shorter texts generate proportionally faster
   - Split long texts for better responsiveness

## 9. Summary and Cleanup

Let's review what we've accomplished and provide final cleanup utilities.

In [ ]:
print("""
Application Features:
1. XPU-optimized inference with automatic mixed precision
2. Model caching to prevent unnecessary reloading
3. Performance tracking and comparison between runs
4. Support for 5 languages with multiple voices
5. Memory management with periodic cleanup
6. Real-time generation feedback
7. Export capabilities for generated audio

Key XPU Optimizations Applied:
- torch.autocast for automatic mixed precision
- torch.no_grad() for inference optimization
- Device synchronization for accurate timing
- Efficient memory management
""")

def check_final_status():
    """
    Check final status of the system.
    """
    print(f"\nFinal Status:")
    print(f"Device: {device.upper()}")
    print(f"Precision: {dtype}")
    print(f"Cached models: {len(MODEL_CACHE)}")
    print(f"Total generations: {GENERATION_COUNT}")
    
    if MODEL_CACHE:
        print("\nCached models:")
        for key in MODEL_CACHE:
            print(f"  - {key}")

# Check final status
check_final_status()

# Uncomment to clear all models and free memory
clear_all_models()

## Conclusion

In this workshop, we've explored how to implement text-to-speech using Bark models with Intel XPU acceleration. We've covered:

1. **XPU Optimization Techniques**: Applied autocast, memory management, and device synchronization
2. **Model Management**: Implemented efficient caching and loading strategies
3. **Interactive Applications**: Built a user-friendly interface with performance tracking
4. **Multilingual Support**: Demonstrated generation in 5 languages
5. **Performance Analysis**: Compared different configurations and optimizations

### Next Steps

To continue your exploration:

1. Experiment with different model sizes and their quality trade-offs
2. Integrate TTS into larger applications (chatbots, reading assistants)
3. Explore voice cloning capabilities with custom voice presets
4. Build REST APIs for TTS services
5. Combine with speech recognition for voice-to-voice translation

The power of local AI processing on Intel AI PCs enables privacy-preserving, low-latency text-to-speech applications that can transform how we create and consume audio content.